In [1]:
from model import *

In [2]:
config = TransformerConfig(vocab_size=30_728)

In [3]:
rngs = nnx.Rngs(0)

In [4]:
model = TransformerLM(config, rngs=rngs)

In [5]:
graphdef, params, rest = nnx.split(model, nnx.Param, ...)

In [9]:
graphdef

NodeDef(
  type=TransformerLM,
  index=0,
  attributes=('config', 'decode', 'decoder'),
  subgraphs={
    'decoder': NodeDef(
      type=Decoder,
      index=1,
      attributes=('config', 'decode', 'dropout', 'encoderdecoder_norm', 'encoderdecoderblock_0', 'encoderdecoderblock_1', 'encoderdecoderblock_2', 'output_embed', 'posembed_out'),
      subgraphs={
        'dropout': NodeDef(
          type=Dropout,
          index=2,
          attributes=('broadcast_dims', 'deterministic', 'rate', 'rng_collection', 'rngs'),
          subgraphs={
            'broadcast_dims': NodeDef(
              type=tuple,
              index=-1,
              attributes=(),
              subgraphs={},
              static_fields={},
              leaves={},
              metadata=None,
              index_mapping=None
            ),
            'rngs': NodeDef(
              type=NoneType,
              index=-1,
              attributes=(),
              subgraphs={},
              static_fields={},
     

In [8]:
rest

State({
  'decoder': {
    'encoderdecoderblock_0': {
      'attention': {
        'rngs': {
          'default': {
            'count': VariableState(
              type=RngCount,
              value=Array(39, dtype=uint32),
              tag='default'
            ),
            'key': VariableState(
              type=RngKey,
              value=Array((), dtype=key<fry>) overlaying:
              [0 0],
              tag='default'
            )
          }
        }
      }
    }
  }
})

In [ ]:
inputs = jax.random.randint(rngs.params(), (2, 128), minval=0, maxval=config.vocab_size)

In [ ]:
transformer(inputs, rngs)

# TESTING

In [ ]:
from flax import nnx
import jax.numpy as jnp

In [ ]:
rngs = nnx.Rngs(0)

In [ ]:
a = nnx.Embed(
    num_embeddings=30_000,
    features=128,
    embedding_init=nnx.initializers.normal(stddev=1.0),
    rngs=rngs,
    )

In [ ]:
import jax

In [ ]:
y = jax.random.uniform(rngs.params(), (128, ))

In [ ]:
a.attend(y) / jnp.sqrt(y.shape[-1])

In [ ]:
inputs = jnp.array([[1, 2, 3, 4, 1, 0, 0, 0, 0], [1, 2, 3, 4, 5, 0, 0, 0, 0] ])

In [ ]:
inputs

In [ ]:
nnx.make_causal_mask(inputs)

In [ ]:
nnx.combine_masks(
  nnx.make_attention_mask(inputs > 0, inputs > 0),
  nnx.make_causal_mask(inputs),
) 